In [1]:
import pandas as pd

splits = {'test': 'plain_text/test-00000-of-00001.parquet', 'validation': 'plain_text/validation-00000-of-00001.parquet', 'train': 'plain_text/train-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/stanfordnlp/snli/" + splits["test"])

/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df

,premise,hypothesis,label
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,1
1,This church choir sings to the masses as they ...,The church is filled with song.,0
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.,2
3,"A woman with a green headscarf, blue shirt and...",The woman is young.,1
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,0
...,...,...,...
9995,Two women are observing something together.,Two women are standing with their eyes closed.,2
9996,Two women are observing something together.,Two girls are looking at something.,0
9997,A man in a black leather jacket and a book in ...,A man is flying a kite.,2
9998,A man in a black leather jacket and a book in ...,A man is speaking in a classroom.,0


In [5]:
# Load the datasets
train_df = pd.read_parquet("hf://datasets/stanfordnlp/snli/" + splits['train'])
test_df = pd.read_parquet("hf://datasets/stanfordnlp/snli/" + splits['test'])
validation_df = pd.read_parquet("hf://datasets/stanfordnlp/snli/" + splits['validation'])

# Create the training dataset with 1000 samples, selecting every 550th sample
train_samples = train_df.iloc[::550].head(1000)

# Create the testing dataset with 100 samples, selecting every 100th sample
test_samples = test_df.iloc[::100].head(100)

# Create the validation dataset with 100 samples, selecting every 100th sample
validation_samples = validation_df.iloc[::100].head(100)

# Show the shapes of the new datasets
print("Training samples shape:", train_samples.shape)
print("Testing samples shape:", test_samples.shape)
print("Validation samples shape:", validation_samples.shape)

# Save the datasets as CSV files
train_samples.to_csv("train_samples.csv", index=False)
test_samples.to_csv("test_samples.csv", index=False)
validation_samples.to_csv("validation_samples.csv", index=False)

Training samples shape: (1000, 3)
Testing samples shape: (100, 3)
Validation samples shape: (100, 3)


In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline


pipe = pipeline("text-generation", model="microsoft/phi-2")

/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.29it/s]
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Check for GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

device_0 = torch.device("cuda:0")  #
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2").to(device_0)

/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.04it/s]


In [1]:
import os
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast

# Set GPU devices to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"  # Specify which GPUs to use

# Check for GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'  # Store the model on GPU 0
print(f"Using device: {device}")

# Load your dataset
train_df = pd.read_csv("train_samples.csv")
validation_df = pd.read_csv("validation_samples.csv")

# Convert DataFrames to Hugging Face Datasets
train_dataset = load_dataset('csv', data_files='train_samples.csv', split='train')
validation_dataset = load_dataset('csv', data_files='validation_samples.csv', split='train')

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2").to(device)  # Load model to GPU 0

# Set pad token to eos token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'],
                     padding="max_length", truncation=True, return_tensors="pt")

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_validation = validation_dataset.map(tokenize_function, batched=True)

# Custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_data):
        self.input_ids = torch.tensor(tokenized_data['input_ids'])
        self.attention_mask = torch.tensor(tokenized_data['attention_mask'])
        self.labels = torch.tensor(tokenized_data['input_ids'])  # For language modeling

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx],
        }

# Create custom datasets
custom_train_dataset = CustomDataset(tokenized_train)
custom_validation_dataset = CustomDataset(tokenized_validation)

# Define QLoRA configuration
lora_config = LoraConfig(
    r=16,  # rank
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)

# Wrap the model with QLoRA
model = get_peft_model(model, lora_config)

# Data Parallelism
model = torch.nn.DataParallel(model)  # Wrap the model for data parallelism

# Training arguments
training_args = TrainingArguments(
    output_dir="./phi2-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_strategy="epoch",
    save_total_limit=5,  # Keep only the last 5 saved models
    report_to="none",  # Disable reporting to avoid errors if not set up
    load_best_model_at_end=True,  # Optionally load the best model at the end of training
)

# Create DataLoader for batching
train_dataloader = DataLoader(custom_train_dataset, batch_size=training_args.per_device_train_batch_size, shuffle=True)
validation_dataloader = DataLoader(custom_validation_dataset, batch_size=training_args.per_device_eval_batch_size)

# Custom training loop with gradient accumulation
scaler = GradScaler()  # Initialize scaler for mixed precision
accumulation_steps = 4  # Adjust as needed
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(training_args.num_train_epochs):
    model.train()
    for i, batch in enumerate(train_dataloader):
        # Move input tensors to the appropriate device (GPU 0 for model, others for inputs)
        inputs = {
            'input_ids': batch['input_ids'].to(device),  # Move to GPU 0
            'attention_mask': batch['attention_mask'].to(device),
            'labels': batch['labels'].to(device)
        }

        with autocast():  # Enable mixed precision
            outputs = model(**inputs)  # Directly use the model
            loss = outputs.loss / accumulation_steps  # Normalize loss for gradient accumulation
        
        scaler.scale(loss).backward()  # Scale loss and backpropagate
        
        if (i + 1) % accumulation_steps == 0:  # Update weights every few batches
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()  # Reset gradients after update

    print(f"Epoch {epoch + 1}/{training_args.num_train_epochs}, Loss: {loss.item()}")

# Save the final model
model.module.save_pretrained("./phi2-finetuned-final")  # Use .module to access the original model for saving
tokenizer.save_pretrained("./phi2-finetuned-final")


/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.16it/s]
/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3653528/4169356760.py:97: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Initialize scaler for mixed precision
/tmp/ipykernel_3653528/4169356760.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Enable mixed precision


OutOfMemoryError: Caught OutOfMemoryError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py", line 96, in _worker
    output = module(*input, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/peft/peft_model.py", line 1859, in forward
    return self.base_model(
           ^^^^^^^^^^^^^^^^
  File "/home/arunb/anaconda3/envs/project1/lib/python3.12/site-packages/torc